In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets

# from pastamarkers import markers

In [ ]:
import webbrowser

def open_website(k):
    website_url = f'https://fink-portal.org/{k}'
    webbrowser.open(website_url)


In [ ]:
import io
import requests

def get_classification(Id):
    r = requests.post(
      'https://fink-portal.org/api/v1/objects',
      json={
        'objectId': Id,
      }
    )

    # Format output in a DataFrame
    pdf = pd.read_json(io.BytesIO(r.content))
        
    
        
    # Get value counts for each classification
    class_counts = pdf['v:classification'].value_counts()

    # Calculate total count
    total_count = class_counts.sum()

    # Calculate percentage for each classification
    class_percentages = (class_counts / total_count * 100).round(1)
    
    # Convert to dictionary
    class_percentages_dict = class_percentages.to_dict()
    print("class :")
    for class_name, percentage in class_percentages_dict.items():
        print(f"          {class_name} : {percentage:.1f}%") 
            
#     return class_percentages_dict


# Retrieve data

We begin by fetching the reduced data using the Python script `data_transfer.py` from the file `df_merged.parquet`, then importing it into Pandas data frame.

In [ ]:
df = pd.read_parquet('../scripts/df_after_upper_vs_valid.parquet', engine='pyarrow')

In [ ]:
df.head(2)

In [ ]:
df.shape

Here we extract all unique IDs from our data frame:

In [ ]:
unique_ids = df['objectId'].unique().tolist()
len(unique_ids)

#

To calculate the weight values \(w_i\), we use the formula: `w_i` =\begin{cases}
\frac{1}{{\sigma_i^2}}, & \text{if data is available for day } i \\
0, & \text{otherwise}
\end{cases}

In [ ]:
missing_data = (df['source'] == 0) | (df['dc_sigflux'] == 0)
df['dc_weight'] = np.where(missing_data, 0, 1 / (df['dc_sigflux'] ** 2))
df['nr_weight'] = np.where(missing_data, 0, 1 / (df['nr_sigflux'] ** 2))

In [ ]:
df[['source','dc_weight']].head(3)

# 

# 

# With distance - factor test 

We group the data by shared ID and create `NumPy` arrays for flux, weighted flux, and the source test(if it's a missing day(data)). We also determine the length of each time series.

In [ ]:
grouped = df.groupby('objectId')

F = grouped['dc_flux'].apply(lambda x: x.values)#.values
sig = grouped['dc_sigflux'].apply(lambda x: x.values)#.values
W = grouped['dc_weight'].apply(lambda x: x.values)#.values
source = grouped['source'].apply(lambda x: x.values)#.values
lengths = grouped['source'].apply(lambda x: len(x))#.values
mjd = grouped['mjd'].apply(lambda x: x.values)#.values


We define the length of our query,window or chunk, along with the limit factor and the size of each window or chunk.

In [ ]:
m=0
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
chunk_size = 2 * (m + 1)

The `'no_match_test'` function evaluates whether there are any matches in the provided array, which contains the source test values for a window. If the sum of the array is less than or equal to 1, indicating that all values are missing or only one value is present, the function returns -99. Otherwise, it returns -1 to initialize the window's status as 'no match'.

In [ ]:
def no_match_test(array):
    if array.sum() <= 1 : 
        return -99 ## all are missing, or all except one
    return -1 # initialise as no match  # can be modifieted ? 

"`objects`" list contains a subset of the objects we intend to work with.

"`L_max`" is defined to facilitate partial iteration, serving as a debugging aid by allowing a limit to be set on the number of iterations performed.

We initialize the NumPy arrays with `None` values.

In [ ]:
objects = unique_ids#[0:3000]
num_objects = len(objects)
# L_max = int(num_objects/2)
L_max = 116
print("L_max ", L_max)

R = np.empty(num_objects, dtype=object)
Matches = np.empty(num_objects, dtype=object)
R_l = np.empty((num_objects, L_max), dtype=object)
num_by_obj = np.zeros((L_max,num_objects), dtype=object)
alp = np.empty((num_objects, L_max), dtype=object)
d = np.empty((num_objects, L_max), dtype=object)

In [ ]:
indexes_objects = range(num_objects)
Q = [None] * (L_max)
num_left_by_Q = [0] * (L_max) ####### we don't need this one anymore <==> sum_i
selected_Q_K_i = np.empty(L_max, dtype=object)

We initialize R using the '`no_match_test`' function.

In [ ]:
for k in indexes_objects:
    
    n = lengths[objects[k]]
    num_chunks = int(n // 2)-m 

    #print(n,num_chunks)

    chunks = np.array([source[objects[k]][i*2 : (i*2+chunk_size)] for i in range(num_chunks)])
    result = np.array(list(map(no_match_test, chunks)))
    R[k] = result.copy()
    
    #Matches[k] = np.zeros(num_chunks, dtype=int)
    
    for l in range(L_max):
        R_l[k][l] = result.copy()
    # a for loop isn't a good idea !!!! 
    


### Loop to compute the distance of the subsequence in the time series to its nearest neighbor.

In [ ]:
start_time = time.time()
rangek2 = list(range(len(objects)))
     
l= 0
while (l < L_max):
    indexes_array = np.array([np.where(array == -1)[0] for array in R], dtype=object)
    rangek2 = [i for i in range(len(objects)) if len(indexes_array[i]) > 0]

    has_empty_list = len(rangek2) == 0
    if has_empty_list:
        print("break , l = ", l )
        break
        
    num_left_by_Q[l] = len(rangek2)
        
    k = rangek2[0]
        
    f = F[objects[k]]
    index_no_match = indexes_array[k][0]
    k_Query_taked = k
    selected_Q_K_i[l] = [k,index_no_match]
    Q[l] = f[index_no_match*2 : index_no_match*2 +chunk_size]


            
    for k in rangek2:
        f = F[objects[k]]
        w = W[objects[k]]
        n = lengths[objects[k]]
        n_c = n - 2*m # (number of chunks x 2) ! it's (n/2 - m) but to optimize we mutiply by 2 directly !  
        #print(n, n_c,len(R[k]))



        s_1 = np.zeros(n_c, dtype=float)
        s_2 = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1): 
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:] += (f[j*2:j*2+ n_c]*h*w[j*2:j*2+ n_c])
            s_2[:] += (h**2 * w[j*2:j*2+ n_c])


        s_n = s_1[::2] + s_1[1::2]  
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        alp[k][l] = np.zeros_like(s_d, dtype=float)

        alp[k][l][mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero

        alpha = np.repeat(alp[k][l], 2) # duplicate alpha for each value (one for r and second for g)
     
    
        dd = np.zeros(n_c, dtype=float)
        
        for j in range(0,m+1):
            h = np.tile(Q[l][j*2: j*2+2], (len(f[j*2:j*2+ n_c]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation
            
            dd[:] += ((f[j*2:j*2+ n_c] - alpha[:] * h)**2) * w[j*2:j*2+ n_c] 
            #alpha[:n-j*2] ==> alpha[:]

        d[k][l] = dd[::2] + dd[1::2]
        
        factor_comparison =  d[k][l] <= factor
                
        R[k][indexes_array[k][factor_comparison[indexes_array[k]]]] = l # explanation follows below!
        
        #count = 0
        for i in range(len(factor_comparison)):
            if factor_comparison[i] and R_l[k][l][i] != -99:
                R_l[k][l][i] = l
                num_by_obj[l][k]+=1
    
    

                

    R_l[k_Query_taked][l][index_no_match] = -2

 
    l += 1 
    
print("l = ",l)



end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

Let's break down the expression `R[k][indexes_array[k][factor_comparison[indexes_array[k]]]] = l` step by step:

1. `indexes_array[k]`: This selects the array of indexes corresponding to the k-th element of `indexes_array`.
2. `factor_comparison[indexes_array[k]]`: This applies boolean indexing to `factor_comparison` using the indexes from `indexes_array[k]`. It selects only the elements of `factor_comparison` corresponding to the indexes in `indexes_array[k]`.
3. `indexes_array[k][factor_comparison[indexes_array[k]]]`: This gives the indices where the condition `factor_comparison` is true for the k-th element of `indexes_array`.
4. `R[k][indexes_array[k][factor_comparison[indexes_array[k]]]]`: This uses the indices obtained in the previous step to select elements from the k-th row of `R`.
5. `= l`: Finally, it assigns the value `l` to the selected elements of `R[k]`.


In [ ]:
num_left_by_Q

In [ ]:
plt.scatter(range(L_max),num_left_by_Q)

plt.xlabel('Queries')
plt.ylabel('Nb objs left')
plt.title('Number of objects left in function of queries.')


# Create DataFrame
data = {'Queries': range(L_max), 'Nb objs left': num_left_by_Q}
df = pd.DataFrame(data)

# Export to Excel
df.to_csv('num_left_by_Q.csv', index=False)

# Show the plot
plt.show()


In [ ]:
rangek2

In [ ]:
R[31518]

In [ ]:
R_l[0]

# 

# 

### Calculate the Matrix of Matches (with windows represented in rows and queries as columns)

In [ ]:
rangek = range(len(objects))

# Initialize list to hold transposed arrays for each k
Matrices_by_k = []

no_missing = np.zeros(len(rangek), dtype= object)
length_no_missing = np.zeros(len(rangek), dtype= int)

Reference_Table_list = []

for k in rangek:
    no_missing[k] = np.where(R[k] != -99)[0]  
    
    length_no_missing[k] = len(no_missing[k])
    
    # Initialize transposed array
    Matrix= np.empty((length_no_missing[k], len(R_l[k])), dtype=object)
    
    refrence_table_k = np.empty(length_no_missing[k], dtype=object)
    
    # Transpose and store the arrays
    for i, idx in enumerate(no_missing[k]):
        for j, arr in enumerate(R_l[k]):
            if arr[idx] == -2 or arr[idx] >= 0:
                Matrix[i][j] = 1
            else:
                Matrix[i][j] = 0
                
        refrence_table_k [i] =  np.array([k ,idx])

                
    # Append transposed array to the list
    Matrices_by_k.append(Matrix)
    Reference_Table_list.append(refrence_table_k)

Matrices_by_k

In [ ]:
Matrix_Matches = np.concatenate(Matrices_by_k, axis=0)
Reference_Table = np.concatenate(Reference_Table_list, axis=0)
Matrix_Matches

In [ ]:
len(Reference_Table),Reference_Table


In [ ]:
def get_i_values_for_k(specific_k):
    return no_missing[specific_k]
get_i_values_for_k(0)  

In [ ]:
def get_k_i_for_row(global_idx):
    return Reference_Table[global_idx]
k,i = get_k_i_for_row(2)
k,i

In [ ]:
def get_i_indexes_for_k(specific_k):
    idx = 0 
    for k in rangek:
        if k == specific_k:
            return list(range(idx, idx + length_no_missing[k]))
        idx += length_no_missing[k]

get_i_indexes_for_k(0)

In [ ]:
sum_j_rows = np.sum(Matrix_Matches, axis=1)
sum_j_rows


In [ ]:
sum_i_columns = np.sum(Matrix_Matches, axis=0)
sum_i_columns

#

#

# Matrix only for queries (len(Qs) x len(Qs))

In [ ]:
Matrix_Queries = []

for k,i in selected_Q_K_i:#[:5]:
    #print(get_i_values_for_k(k),i)
    i_k = np.where(get_i_values_for_k(k) == i)[0][0]
    #print(get_i_indexes_for_k(k),i_k)
    global_idx = get_i_indexes_for_k(k)[i_k]
    Matrix_Queries.append(Matrix_Matches[global_idx])

    #[0][0]
Matrix_Queries

In [ ]:
len(Matrix_Queries)

In [ ]:
sum_j_rows_queries = np.sum(Matrix_Queries, axis=1)

sum_i_columns_queries = np.sum(Matrix_Queries, axis=0)
sum_i_columns_queries,sum_j_rows_queries

In [ ]:
sum_i_columns_queries.argmax(),sum_i_columns_queries.max()

In [ ]:
sum_j_rows_queries.argmax(), sum_j_rows_queries.max()

In [ ]:
sum_j_rows_queries.sum(), sum_i_columns_queries.sum()

#

#

In [ ]:
total_matches = sum(sum_i_columns) + sum(sum_j_rows)

I_i_j = Matrix_Matches[:].copy() #*(sum_i[:])
I_i_j


In [ ]:
for j, sj in enumerate(sum_j_rows):
    for i, si in enumerate(sum_i_columns):
        dn = (si*sj)
        if dn == 0:
            I_i_j[j][i] *= np.inf # we don't need this anymore , No, we do , if l_max != len(Q)
        else:
            I_i_j[j][i] *= total_matches/ dn
I_i_j

In [ ]:
# nonzero_indices = np.nonzero(np.outer(sum_j, sum_i))

# # Update only non-zero elements
# I_i_j[nonzero_indices] = I_i_j[nonzero_indices] / (sum_i[nonzero_indices[1]] * sum_j[nonzero_indices[0]])

# #I_i_j[I_i_j == 0] = np.inf


#

###### Compute the query with the highest number of matches and the one with the fewest matches.

In [ ]:
max(sum_i_columns),min(sum_i_columns), np.argmax(sum_i_columns), np.argmin(sum_i_columns), selected_Q_K_i[94], selected_Q_K_i[100]

###### Compute the window with the highest number of matches on queries and the one with the fewest matches.

In [ ]:
#flattened_array = np.concatenate([arr.flatten() for arr in sum_j])
maximum_value = np.max(sum_j_rows)
maximum_value, np.argmax(sum_j_rows),sum_j_rows[155]

In [ ]:
# Find indices of non-zero values
non_zero_indices = np.argwhere(sum_j_rows != 0).flatten()

# Retrieve non-zero values
non_zero_values = sum_j_rows[non_zero_indices]

# Find the index of the minimum non-zero value
min_non_zero_index = non_zero_indices[np.argmin(non_zero_values)]

min_non_zero_value = sum_j_rows[min_non_zero_index]

min_non_zero_value, min_non_zero_index

# 

### Function to detect a specific window in a time series (specified by its object and index within this object).

In [ ]:
def if_a_query(k_TS, i_TS, selected_Q_K_i):
    mask = selected_Q_K_i != None
    for index_Q, (k, i) in enumerate(selected_Q_K_i[mask]):
        if k_TS == k and i_TS == i:
            return True, index_Q, f"This window is used as the {index_Q}th query! "
    return False, 0 , "This window is not used as a query ! "

In [ ]:
def get_m(i,j):
    return concatenated_array[j][i] ### I need to check this 2 j,i or i,j

def get_i_by_sum(i):
        return sum_i_columns[i]

def get_j_by_sum(j):
        return sum_i_columns[j]

    
def get_k_i_of_Q(l):
    return selected_Q_K_i[l]


In [ ]:
# a_query, l , string = if_a_query(22,5,selected_Q_K_i)
# plot_distance_flux(22,l)

#

#

#

In [ ]:
# import ipywidgets as widgets

# Define a function to be called when the button is clicked
def on_button_click(b):
    print("Button clicked")

# Create a button widget
button = widgets.Button(description="Click me!")

# Connect the button widget to the function
button.on_click(on_button_click)

# Display the button
display(button)


#

#

## Functions to compute the numbers of matches and plot the results 

#### these functions should be in another file

In [ ]:
def plot_distance_flux(k,l,open_Fink= False):
    print("\n______________________________________________________________________________________________")
    print(f"{objects[k]}")
    get_classification(objects[k])
    
#     for class_name, percentage in dict_classes.items():
#         print(f"                        {class_name} : {percentage:.1f}%") 
    import ipywidgets as widgets
          
    button = widgets.Button(description=f"open in FINK")
    # Capture current values of kk and l in lambda's default arguments
    button.on_click(lambda _, k=k : on_button_clicked_plot(objects[k]))
    display(button)

    
    plt.figure(figsize=(14, 12))

    # Plot for F[k]
    plt.subplot(2, 1, 1)

    for i in range(int(len(F[objects[k]])/2)):
        if source.loc[objects[k]][2*i] == 0:
            marker = 'x'
            plt.errorbar(mjd.loc[objects[k]][2*i], F.loc[objects[k]][2*i], 
                     #sig[objects[k]][2*i]*0.4,
                     c='C0', marker='x')
        else:
            marker = 'o'
            plt.errorbar(mjd.loc[objects[k]][2*i], F.loc[objects[k]][2*i], 
                     sig.loc[objects[k]][2*i],
                     c='C0', marker='o')

        if source.loc[objects[k]][2*i+1] == 0:
            marker = 'x'
            plt.errorbar(mjd.loc[objects[k]][2*i+1], F.loc[objects[k]][2*i+1],
                     #sig[objects[k]][2*i+1]*0.4,
                     c='C1', marker='x')
            
        else:
            marker = 'o'
            plt.errorbar(mjd.loc[objects[k]][2*i+1], F.loc[objects[k]][2*i+1],
                     sig.loc[objects[k]][2*i+1],
                     c='C1', marker='o')
            
        a_query,l_1, string = if_a_query(k,i,selected_Q_K_i)
            
        if a_query and l_1 == l:
            print(string)
            
            # Define the window of indices
            window_start = mjd.loc[objects[k]][2*i]  # Index of the window start
            window_end = mjd.loc[objects[k]][2*i] + int(len(Q[l])/2 - 1)  # Index of the window end
            
            
            # Create an array of float indices
            indices = np.arange(window_start, window_end + 1)
            indices = np.concatenate(([indices[0] - 0.5], indices, [indices[-1] + 0.5]))

            # Plot a shaded region for the window
            plt.fill_between(indices, min(Q[l])/1.2, max(Q[l])*1.2, color='gray', alpha=0.2)
            
        
    plt.plot([], [], color='C1', marker='x', label='missing points !')
    plt.plot([], [], color='C0', marker='o', label='origin')
    plt.plot([], [], color='C0', marker='x', label='missing points !')
    plt.plot([], [], color='C1', marker='o', label='origin')
    
   
    ################################### what about the seccond window !? 
    moy_alp = 1+0*alp[k][l].mean()
    #moy_alp = np.median(F[k])/np.median(Q[l]) #Manu test

    plt.plot(mjd.loc[objects[k]][::2], F.loc[objects[k]][::2], c='C0', linewidth = 1)
    plt.plot(mjd.loc[objects[k]][1::2], F.loc[objects[k]][1::2], c='C1', linewidth = 1)

    plt.plot(np.arange(len(Q[l]) // 2) - 1-m + mjd.loc[objects[k]][0], Q[l][::2]*moy_alp, c='g', label='Q[l]',marker='.', linewidth=3, zorder=3)
    plt.plot(np.arange(len(Q[l]) // 2) - 1-m + mjd.loc[objects[k]][0], Q[l][1::2]*moy_alp, c='r', label='Q[l]',marker='.', linewidth=3, zorder=3)

    # Define the window of indices
    window_start = -1-m + mjd.loc[objects[k]][0] # Index of the window start
    window_end = int(len(Q[l])/2 - 1-1 -m)+ mjd.loc[objects[k]][0]  # Index of the window end

    # Create an array of float indices
    indices = np.arange(window_start, window_end + 1)
    indices = np.concatenate(([indices[0] - 0.4], indices, [indices[-1] + 0.4]))

    # Plot a shaded region for the window
    plt.fill_between(indices, min(Q[l])/1.2*moy_alp, max(Q[l])*1.2*moy_alp, color='gray', alpha=0.2)

    # plt.xlabel('Index')
    plt.ylabel('Flux')
    # plt.title('Flux Plot')
    plt.legend()
    




#     ###############                       Plot for d[k][0]                       ###############

    plt.subplot(2, 1, 2)
    plt.plot([-1-m, -1-m], [0, 0], color='none')  # Plot an empty line with zero length
    plt.plot(range(len(d[k][l])), d[k][l],color='C0', linestyle='-',linewidth=1)
    
#     matches = all_matches[k]#Matches[k]#
#     for i, match_count in enumerate(matches):
#         plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(match_count), fontsize=10, color='red', ha='left')
    matches_idx = get_i_indexes_for_k(k)
    matches2 = sum_j_rows[matches_idx]

    # Plot dummy points with desired colors and markers
    plt.plot([], [], color='black', marker='o', label='Query chosed')#markers.soli
    plt.plot([], [], color='red', marker='x', label='missing cases !')
    plt.plot([], [], color='blue', marker='s', label='Matched here')#markers.ravioli
    plt.plot([], [], color='green', marker='*', label='Matches with a different `l` (Query)')#markers.stelline
    plt.plot([], [], color='yellow', marker='^', label=f'Not matched with any of the {L_max} options we selected') #markers.tortellini
    
    c= 0
    
    for i, val in enumerate(R_l[k][l]):
            if val == -99:
                plt.scatter(i, d[k][l][i], color='red', marker='x', s=50)  # marker size 50
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, 0, fontsize=10, color='red', ha='left')
                c+=1
#                 plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, 0*str(matches2[i]), fontsize=10, color='red', ha='left')

            elif val == l:
                plt.scatter(i, d[k][l][i], color='blue', marker='s', s=50)
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i-c]), fontsize=10, color='blue', ha='left')

            elif val == -2:
                plt.scatter(i, d[k][l][i], color='black', marker='o', s=50) 
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i-c]), fontsize=10, color='black', ha='left')

            elif matches2[i-c]==0:
                plt.scatter(i, d[k][l][i], color='yellow', marker='^', s=50) 
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i-c]), fontsize=10, color='yellow', ha='left')

            else:
                plt.scatter(i, d[k][l][i], color='green', marker='*', s=75)
                plt.text(i+0.2, d[k][l][i]+d[k][l][i]*5/100, str(matches2[i-c]), fontsize=10, color='green', ha='left')




    plt.legend(fontsize=8) 
    #plt.legend(loc='upper right', fontsize=8)

    plt.ylabel('distance')



    ###############                       Plot for alpha[k][l]                       ###############

#     plt.subplot(3, 1, 3)
#     plt.plot(range(len(alp[k][l])), alp[k][l], marker='.', linestyle='-',color='black')
#     plt.xlabel('Index')
#     plt.ylabel('Value')
#     plt.title('alpha Plot')




    plt.subplots_adjust(top=0.93)  # Adjust the top margin for the super title
    plt.suptitle(f"{objects[k]}, k = {k}, l = {l} ", fontname='Arial', fontsize=16, fontweight='bold')
    plt.show()
    

    if open_Fink:
        open_website(objects[k])


def on_button_clicked_plot(Id):
    print("URL opened")
    open_website(Id)


In [ ]:
plot_distance_flux(0,0)


# 



# 



# 


# plot by multiple l

In [ ]:
# for l in [0,1]:
#     for k in [0,1]:
#         plot_distance_flux(k,l)

In [ ]:
# l_values = [100,0]#range(119)
# k_values=range(len(objects))
# k=1
# for l in l_values:
#     for k in k_values[:7]:
#         plot_distance_flux(k,l) # True == open website


#

## plot histogram of numbers of matches for each Q

We graph the number of matches for each query.

In [ ]:
indexes = range(len(sum_i_columns))

# Plotting the histogram
plt.hist(indexes, bins=len(sum_i_columns), weights=sum_i_columns, color='C1', edgecolor='white')

# Adding labels and title
plt.xlabel('Indices')
plt.ylabel('Numbers')
plt.title('Histogram of numbers of matches for each Q ')

# Display the histogram
plt.show()


# Compute the highest and lowest 10 values and their indexes
sorted_indices = sorted(range(len(sum_i_columns)), key=lambda i: sum_i_columns[i])
lowest_10_indices = sorted_indices[:20]
highest_10_indices = sorted_indices[-10:][::-1]  # Reverse to get in descending order

total_windows = len(Matrix_Matches)

# Print the highest and lowest 10 values and their indexes with percentages
print("Highest 10:\n")
for i in highest_10_indices:
    percentage = (sum_i_columns[i] / total_windows) * 100
    print(f"Index: {i}, Matches: {sum_i_columns[i]}, Percentage: {percentage:.2f}%")

print("\n\nLowest 10:\n")
for i in lowest_10_indices:
    percentage = (sum_i_columns[i] / total_windows) * 100
    print(f"Index: {i}, Matches: {sum_i_columns[i]}, Percentage: {percentage:.2f}%")

    
    


In [ ]:
import ipywidgets as widgets

def on_button_clicked(k,l):
    print("Button clicked:")
    plot_distance_flux(k,l)

# Create the button widget


In [ ]:
for l in sorted_indices[14:20]:
    print(f"Q[{l}] , Matches on this Query: {sum_i_columns[l]}")
    k , i = selected_Q_K_i[l]
    print("Ratio :",Q[l][0]/Q[l][1])
    
    print(f"Object of this Q : {objects[k]} : https://fink-portal.org/{objects[k]}")

    
    non_zero_indexes = np.nonzero(num_by_obj[l])[0]
    non_zero_indexes, len(non_zero_indexes)
    print("\nNumber of objects that have matches on this Q :", len(non_zero_indexes))
    print("Objects that have matches on this Q : ", non_zero_indexes,"\n")
    
    for kk in non_zero_indexes:
        print(f"https://fink-portal.org/{objects[kk]}" , " : Nb of mathces : ",num_by_obj[l][kk])
        button = widgets.Button(description=f"plot {objects[kk]}")
        # Capture current values of kk and l in lambda's default arguments
        button.on_click(lambda _, kk=kk, l=l: on_button_clicked(kk, l))
        display(button)
        
    print()
    
    plot_distance_flux(k,l)
    

In [ ]:
plot_distance_flux(25337,111)

In [ ]:
F[objects[18039]]-F[objects[25337]]


In [ ]:
F[objects[25337]]

In [ ]:
import pandas as pd

# Assuming you have already imported numpy as np

# Create a list to store dictionaries with the information
data = []

for l in sorted_indices[0:20]:
    Q_ratio = Q[l][0] / Q[l][1]
    k, i = selected_Q_K_i[l]
    non_zero_indexes = np.nonzero(num_by_obj[l])[0]
    
    url_and_matches = []
    for kk in non_zero_indexes:
        url_and_matches.append(f"https://fink-portal.org/{objects[kk]} : Nb of matches : {num_by_obj[l][kk]}")
    
    # Create a dictionary for each iteration
    row = {
        'Query': l,
        'Matches on this Query': sum_i_columns[l],
        'Ratio': Q_ratio,
        'Object of this Q': objects[k],
        'Link': f"https://fink-portal.org/{objects[k]}",
        'Number of objects': len(non_zero_indexes),
        'Objects with matches': non_zero_indexes.tolist(),
        'URLs and Matches': '\n'.join(url_and_matches)
    }
    
    # Append the dictionary to the data list
    data.append(row)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Export DataFrame to Excel
df.to_excel('lowest_Q_EM.xlsx', index=False)


In [ ]:
objects[k], k , objects[25337]

In [ ]:
F[objects[18039]]

In [ ]:
F[objects[25337]]

In [ ]:
import csv

# Specify the file name
file_name = "Num_for_Qs_EXM.csv"

# Combine the data into a list of tuples
data = list(zip(indexes, sum_i_columns))

# Write the data to a CSV file
with open(file_name, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write a header if needed
    csv_writer.writerow(['Queries', 'Nb of matches'])
    
    # Write the data
    csv_writer.writerows(data)

print("Data has been written to", file_name)


We calculate the minimum and maximum number of matches, along with their corresponding indexes (where index represents the index of the query).

In [ ]:
max(sum_i_columns),min(sum_i_columns),np.argmax(sum_i_columns), np.argmin(sum_i_columns)

#

#

### plot Qr/Qg 

In [ ]:
Q_array = np.array(Q)
ratios = Q_array[:, 0] / Q_array[:, 1]

plt.plot(ratios)
plt.title('Ratio of First Value to Second Value')
plt.xlabel('Index')
plt.ylabel('Ratio')
plt.grid(True)
plt.show()

### Here, we plot the number of matches as a function of the ratios (Qr/Qg).

In [ ]:
Q_array = np.array(Q)
ratios = Q_array[:, 0] / Q_array[:, 1]

plt.scatter(ratios[:-3],sum_i_columns[:-3]) ###################### -3 to exclude the 800 .. 
plt.title('num of matches for each Q in function of Ratio ')
plt.xlabel('Ratio Qr/Qg')
plt.ylabel('num(l)')
plt.grid(True)
plt.yscale('log')
plt.show()


print("\nHighest 10 Ratios, Matches, and Indices:\n")
for i in range(10):
    index = highest_10_indices[i]
    ratio = ratios[index]
    matches = sum_i_columns[index]
    print("Index:", index, "Ratio:", ratio, "Matches:", matches)
    
print("\n\nLowest 10 Ratios, Matches, and Indices:\n")
for i in range(10):
    index = lowest_10_indices[i]
    ratio = ratios[index]
    matches = sum_i_columns[index]
    print("Index:", index, "Ratio:", ratio, "Matches:", matches)


In [ ]:
selected_Q_K_i[64], objects[854],plot_distance_flux(854, 64, True)

In [ ]:
import csv

# Specify the file name
file_name = "ratios_Q_119_EXM.csv"

# Combine the data into a list of tuples
data = list(zip(ratios, sum_i_columns))

# Write the data to a CSV file
with open(file_name, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write a header if needed
    csv_writer.writerow(['Ratio', 'Sum of i Columns'])
    
    # Write the data
    csv_writer.writerows(data)

print("Data has been written to", file_name)


#

#

### Compute the highest and the lowest indices 

In [ ]:
# lowest_indices_array = []
sorted_queries_list = []


for l in range(len(num_by_obj)):

    array = np.array(num_by_obj[l])

    # Get indices of sorted array in ascending order
    sorted_queries = np.argsort(array)
    sorted_queries_list.append(sorted_queries)  # Append sorted indices list

#     # Indices of 10 lowest values

#     lowest_values = array[sorted_indices[0]]
#     all_lowest_indices = np.where(array == lowest_values)[0]
#     lowest_indices_array.append(all_lowest_indices)


#

#

In [ ]:
num_by_obj[112]

In [ ]:
# for k in rangek:
#     print(np.sum(Matrices_by_k[k], axis=0))

## plot histogram of numbers of matches for all objects with a specific Q

In [ ]:
l=108
indexes = range(len(num_by_obj[l]))

# Plotting the histogram
plt.hist(indexes, bins=len(num_by_obj[l]), weights=num_by_obj[l], color='C1', edgecolor='black')
plt.xlabel('Objects')
plt.ylabel('Numbers')
plt.title('Histogram of numbers of matches for a specific Q for all objects')
plt.show()



total_queries = len(Q)
lowest_10_indices = sorted_queries_list[l][:10]


array = np.array(num_by_obj[l])
lowest_value = array[sorted_queries_list[l][0]]
all_lowest_indices = np.where(array == lowest_value)[0]

lowest_10_indices_1 = sorted_queries_list[l][len(all_lowest_indices):len(all_lowest_indices)+10]
# These aren't simply the lowest 10 values; rather,
#they represent the lowest 10 values with the condition that their number of matches exceeds
#that of the lowest value. Suppose the lowest value of matches is 0, these objects are those with more than 0 matches.
# (the lowest ones with Nb matches > 0  ).


highest_10_indices = sorted_queries_list[l][-10:][::-1]  # Reverse to get in descending order

# Print the highest and lowest 10 values and their indexes with percentages
print(f"Q[{l}]:\n")
print("Highest 10:\n")
for k in highest_10_indices:
    percentage = (num_by_obj[l][k] / total_queries) * 100
    print(f"Index: {k}, Id : {objects[k]}, Matches: {num_by_obj[l][k]}, Percentage: {percentage:.2f}%")

print("\nLowest 10:\n")
for k in lowest_10_indices:
    percentage = (num_by_obj[l][k] / total_queries) * 100
    print(f"Index: {k}, Id : {objects[k]}, Matches: {num_by_obj[l][k]}, Percentage: {percentage:.2f}%")

print("\nLowest 10 with Nb matches > (lowest Nb):\n")
for k in lowest_10_indices_1:
    percentage = (num_by_obj[l][k] / total_queries) * 100
    print(f"Index: {k}, Id : {objects[k]}, Matches: {num_by_obj[l][k]}, Percentage: {percentage:.2f}%")


In [ ]:
non_zero_indexes = np.nonzero(num_by_obj[108])[0]
num_by_obj[108][non_zero_indexes], non_zero_indexes

In [ ]:
for k in highest_10_indices[8:9]:
    plot_distance_flux(k, 108,True)

#

In [ ]:
k= 100 
for k in rangek:
    print(k, num_by_obj[l][k]/length_no_missing[k]*100)

In [ ]:
num_by_obj[l][136], count_missing_windows[136], len(R[136])

In [ ]:
k= 864 
num_by_obj[l][k],length_no_missing[k], count_missing_windows[k], len(R[k])

In [ ]:
plot_distance_flux(k,l)

In [ ]:
lensss = list(map(len, R))

lensss  

### We restrict the selection of objects to those with a specific count of missing windows! 
Therefore, we prioritize objects with the highest number of intact windows.

In [ ]:
count_missing_windows = np.array([np.sum(subarr == -99) for subarr in R])

result = np.where(count_missing_windows <=10, True, False)
# The result array represents the condition for each object (i.e., objects with few missing windows).
result,len(result),count_missing_windows

We extract the indices where the condition is true, representing the objects with the highest numbers of normal data.

In [ ]:
indices = np.where(result)[0]
l = 0
selected_arrays = num_by_obj[l][indices]
selected_arrays, indices, len(selected_arrays)

We indentify and plot the object with the highest number of matches for a query "l".

In [ ]:
k = indices[np.argmax(selected_arrays)]
k

In [ ]:
mask = selected_Q_K_i != None
for index_Q, (k1, i) in enumerate(selected_Q_K_i[mask]):
    if 17435 == k1 :
        print(f"This window is used as the {index_Q}th query! ")
        break

In [ ]:
plot_distance_flux(1,l), print(objects[k])

###

### Generate a plot illustrating the match counts of the selected objects(Limited to missing window counts.) for a particular query, Q.

In [ ]:
l=0
indexes = range(len(num_by_obj[l][indices]))

# Plotting the histogram
plt.hist(indexes, bins=len(num_by_obj[l][indices]), weights=num_by_obj[l][indices], color='skyblue', edgecolor='C0')

# Adding labels and title
plt.xlabel('Indices')
plt.ylabel('Numbers')
plt.title('Histogram of numbers of matches for a specific Q for a limited nb of objects')

# Display the histoazgram
plt.show()

##

#

#

### plot the highest and the lowest indices 

In [ ]:
l=94
count = 1
for k in sorted_indices_list[l][len(lowest_indices_array[l]):len(lowest_indices_array[l])+10]:
    print(count)
    count += 1
    print(objects[k],k,"l =",l,".")
    print("Number of matches in this object of this Q",num_by_obj[l][k])
    plot_distance_flux(k,l,True)
    

### 

### 

# Negative Flux ! 

In [ ]:
negatives = []
k_negatives = []
for k in rangek:
    if np.any(F[objects[k]] <0):
        k_negatives.append(k)
        negative_mask = F[objects[k]] < 0 
        negatives.append(F[objects[k]][negative_mask] / sig[objects[k]][negative_mask])

negatives,k_negatives

In [ ]:
for idx, list1 in enumerate(negatives):
    #print(list1)
    if np.any(np.array(list1) <-3):
        print(F[objects[k_negatives[idx]]]/sig[objects[k_negatives[idx]]])
        plot_distance_flux(k_negatives[idx],0,True)

In [ ]:
F[objects[5053]]/sig[objects[5053]]

In [ ]:

ravel = np.concatenate(negatives)

In [ ]:
plt.hist(ravel,bins='fd')

In [ ]:
len(ravel)

In [ ]:
for k in k_negatives[:3]:
    plot_distance_flux(k,0)

### 

### 

### 

#

#

#

#

#

#

# TEST

#